In [6]:
traindata = [
    [1,2,0,2,1],
    [4,6,1,3,7]
]
trainlabel = [1,1,-1,-1,1]
testdata = [
    [1,2,0,2,1],
    [4,6,1,3,7]
]
testlabel = [1,1,-1,-1,1]
nBayesClassifier(traindata, trainlabel, testdata, testlabel, 0)

([1, 1, -1, 1, 1], 0.75, 1.0, 0.8571428571428571)

In [77]:
def nBayesClassifier(traindata, trainlabel,testdata, testlabel, threshold):
    # 朴素贝叶斯分类器
    # threshold 为用于判断类别的后验概率的阈值。
    # 要求函数返回对测试数据的预测 ypred，以及通过与 ground label(真实标签)比较计算得到的性能指标 SP,SR 和 F。
    # ypred 与 SP,SR,F 以 tuple 形式返回。我们要求以表格的形式给出你的结果(其余两个分类器要求类似)并对分类器性能做出分析。
    # SP: 垃圾邮件识别准确率, Precision
    # SR: 垃圾邮件识别的查全率, Recall
    # F = SP*SR*2/(SP+SR)
    traindata = np.mat(traindata)
    testdata = np.mat(testdata)
    trainlabel = np.array(trainlabel)
    testlabel = np.array(testlabel)
    print("begin training...")
    wordNum = len(traindata)
    trainsetLen = len(trainlabel)
    testsetLen = len(testlabel)
    pIfSpam = [0 for i in range(wordNum)]
    pIfNotSpam = [0 for i in range(wordNum)]
    pWord = [0 for i in range(wordNum)]
    # 求先验概率
    countSpam = 0
    for i in range(trainsetLen):
        if trainlabel[i]==1:
            countSpam = countSpam + 1
    # countSpam为训练集中的垃圾邮件数
    pSpam = countSpam/(trainsetLen)
    countEasyHam = trainsetLen - countSpam
    pNotSpam = 1 - pSpam
    # 计算 P(wordi|Spam) 和 P(wordi|EasyHam) 和 P(wordi)
    for i in range(trainsetLen):
        #print(i)
        if(trainlabel[i]==1):
            # Spam
            for j in range(wordNum):
                if(traindata[j,i]>threshold):
                    pIfSpam[j] = pIfSpam[j] + 1
                    pWord[j] = pWord[j] + 1
        else:
            # Easy Ham
            for j in range(wordNum):
                if(traindata[j,i]>threshold):
                    pIfNotSpam[j] = pIfNotSpam[j] + 1 
                    pWord[j] = pWord[j] + 1 
    for i in range(wordNum):
        pIfNotSpam[i] = pIfNotSpam[i] / countEasyHam
        pIfSpam[i] = pIfSpam[i] / countSpam
        pWord[i] = pWord[i] / trainsetLen
    # pIfNotSpam[i]: P(wordi|Easy Ham)
    # pIfSpam[i]: p(wordi|Spam)
    print("begin testing...")
    # 测试
    nPam2Pam = 0
    nNormal2Pam = 0
    nPam = 0
    ypred = [0 for i in range(testsetLen)]
    for j in range(testsetLen):
        #print(j)
        p1 = pSpam
        p2 = pNotSpam
        for i in range(wordNum):
            if(testdata[i,j]>threshold):
                if(pWord[i]==0):
                    continue
                # Spam, have wordi
                p1 = p1 * pIfSpam[i]
                p1 = p1 / pWord[i]
                # Not Spam, have wordi
                p2 = p2 * pIfNotSpam[i]
                p2 = p2 / pWord[i]
            else:
                if(pWord[i]==1):
                    continue
                # Spam, not have wordi
                p1 = p1 * (1-pIfSpam[i])
                p1 = p1 / (1-pWord[i])
                # Not Spam, not have wordi
                p2 = p2 * (1-pIfNotSpam[i])
                p2 = p2 / (1-pWord[i])
        if(p1>=p2):
            ypred[j] = 1
            # predict spam
            if(testlabel[j]==1):
                # Spam
                nPam2Pam = nPam2Pam + 1
                nPam = nPam + 1
            else:
                #Normal
                nNormal2Pam = nNormal2Pam + 1
        else:
            if(testlabel[j]==1):
                nPam = nPam + 1
            ypred[j] = -1
    SP = nPam2Pam/(nPam2Pam+nNormal2Pam)
    SR = nPam2Pam/nPam
    F = SP*SR*2/(SP+SR)
    return ypred, SP, SR, F

In [18]:
import os
import re
import sys
import random
'''
droprate = 0.8

def textParse(line, vocab):
    words = re.split(r'[_\W*]',line)
    #return [tok.lower() for tok in words if len(tok)>2]
    for i in words:
        i = i.lower()
        rand = random.random()
        if(i.isalpha() and len(i)<=15 and len(i)>2):
            if(rand>droprate):
                vocab.add(i)
    return vocab   

filepath = "C:\\Users\\weiq6\\Desktop\\20021010_easy_ham\\easy_ham"
pathDir =  os.listdir(filepath)
i = 0
for allDir in pathDir:
    if(i<250):
        i = i + 1
        continue
    path = os.path.join('%s\\%s' % (filepath, allDir))
    f = open(path,"r",encoding="windows-1252")
    #print(path)
    header = True
    for line in f:
        if(line=="\n"):
            header = False
        if(header==False):
            vocab = textParse(line,vocab)
    i = i + 1
    f.close()
'''
vocab = set()
filepath = "C:\\Users\\weiq6\\Desktop\\20021010_spam\\spam"
pathDir =  os.listdir(filepath)
i = 0
for allDir in pathDir:
    if(i<50):
        i = i + 1
        continue
    path = os.path.join('%s\\%s' % (filepath, allDir))
    f = open(path,"r",encoding="windows-1252")
    header = True
    #print(path)
    try:
        for line in f:
            if(line=="\n"):
                header = False
            if(header==False):
                vocab = textParse(line,vocab)
    except:
        print("error at "+path)
    i = i + 1
    f.close()


vocab = list(vocab)
print(len(vocab))
print(vocab)

error at C:\Users\weiq6\Desktop\20021010_spam\spam\0123.68e87f8b736959b1ab5c4b5f2ce7484a
error at C:\Users\weiq6\Desktop\20021010_spam\spam\0273.51c482172b47ce926021aa7cc2552549
error at C:\Users\weiq6\Desktop\20021010_spam\spam\0330.a4df526233e524104c3b3554dd8ab5a8
error at C:\Users\weiq6\Desktop\20021010_spam\spam\0334.3e4946e69031f3860ac6de3d3f27aadd
error at C:\Users\weiq6\Desktop\20021010_spam\spam\0335.9822e1787fca0741a8501bdef7e8bc79
5979
['manhattan', 'aboutus', 'secrets', 'next', 'reward', 'structure', 'edition', 'participation', 'european', 'holder', 'pipe', 'freeinfo', 'care', 'journées', 'sequence', 'planovima', 'salviae', 'expand', 'district', 'playing', 'occupying', 'gov', 'yourfreepresent', 'für', 'insider', 'fruitless', 'enctype', 'imposed', 'janm', 'paper', 'êáï', 'select', 'pnhe', 'determined', 'employ', 'front', 'cdtcpijlmjkgonf', 'trouverez', 'images', 'staggering', 'franchise', 'dot', 'êòáòã', 'senior', 'duv', 'selling', 'shape', 'patron', 'girls', 'respon', 'magic

In [19]:
# 设置200个EasyHam和50个Spam邮件作为测试集
wordNum = len(vocab)

filepath = "C:\\Users\\weiq6\\Desktop\\20021010_easy_ham\\easy_ham"
pathDir =  os.listdir(filepath)
len1 = len(pathDir)
filepath = "C:\\Users\\weiq6\\Desktop\\20021010_spam\\spam"
pathDir =  os.listdir(filepath)
len2 = len(pathDir)
testsetLen = 300
trainsetLen = len1 + len2 - testsetLen

testdata = [[0 for i in range(testsetLen)] for j in range(wordNum)]
testlabel = [1 if i<250 else -1 for i in range(testsetLen)]
traindata = [[0 for i in range(trainsetLen)] for j in range(wordNum)]
trainlabel = [1 if i<(len1-250) else -1 for i in range(trainsetLen)]
# 先easy ham，后spam

def Parse(line,i,testNum,base):
    words = re.split(r'[_\W*]',line)
    #return [tok.lower() for tok in words if len(tok)>2]
    for word in words:
        word = word.lower()
        if(word.isalpha() and len(word)<=15):
            if word in vocab:
                index = vocab.index(word)
                if(index>=0):
                    if(i<testNum):
                        testdata[index][base*250+i] += 1
                    else:
                        traindata[index][base*(len1-250)+i-testNum] += 1


        
print("processing easy ham set...")
filepath = "C:\\Users\\weiq6\\Desktop\\20021010_easy_ham\\easy_ham"
pathDir =  os.listdir(filepath)
i = 0
for allDir in pathDir:
    path = os.path.join('%s\\%s' % (filepath, allDir))
    f = open(path,"r",encoding="windows-1252")
    #print(path)
    header = True
    for line in f:
        if(line=="\n"):
            header = False
        if(header==False):
            Parse(line,i,250,0)
    i = i + 1
    f.close()

print("processing spam set...")
filepath = "C:\\Users\\weiq6\\Desktop\\20021010_spam\\spam"
pathDir =  os.listdir(filepath)
i = 0
for allDir in pathDir:
    path = os.path.join('%s\\%s' % (filepath, allDir))
    f = open(path,"r",encoding="windows-1252")
    header = True
    #print(path)
    try:
        for line in f:
            if(line=="\n"):
                header = False
            if(header==False):
                Parse(line,i,50,1)
    except:
        print("error at "+path)
    i = i + 1
    f.close()


processing easy ham set...
processing spam set...
error at C:\Users\weiq6\Desktop\20021010_spam\spam\0123.68e87f8b736959b1ab5c4b5f2ce7484a
error at C:\Users\weiq6\Desktop\20021010_spam\spam\0273.51c482172b47ce926021aa7cc2552549
error at C:\Users\weiq6\Desktop\20021010_spam\spam\0330.a4df526233e524104c3b3554dd8ab5a8
error at C:\Users\weiq6\Desktop\20021010_spam\spam\0334.3e4946e69031f3860ac6de3d3f27aadd
error at C:\Users\weiq6\Desktop\20021010_spam\spam\0335.9822e1787fca0741a8501bdef7e8bc79


In [20]:
nBayesClassifier(traindata, trainlabel, testdata, testlabel, 0)

begin training...
begin testing...


([1,
  1,
  1,
  -1,
  1,
  1,
  1,
  -1,
  -1,
  -1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  -1,
  -1,
  1,
  1,
  1,
  1,
  1,
  -1,
  -1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  -1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  -1,
  1,
  1,
  1,
  -1,
  1,
  1,
  -1,
  1,
  1,
  1,
  -1,
  -1,
  -1,
  -1,
  1,
  1,
  -1,
  1,
  1,
  1,
  -1,
  1,
  1,
  -1,
  -1,
  1,
  1,
  -1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  -1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  -1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  -1,
  1,
  -1,
  1,
  -1,
  1,
  -1,
  1,
  1,
  1,
  -1,
  1,
  -1,
  1,
  -1,
  -1,
  1,
  1,
  1,
  1,
  -1,
  1,
  -1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  -1,
  -1,
  1,


In [22]:
for i in range(20):
    ypred, SP, SR, F = nBayesClassifier(traindata, trainlabel, testdata, testlabel, i)
    print(SP, SR, F)

begin training...
begin testing...
0.9948979591836735 0.78 0.874439461883408
begin training...
begin testing...
0.952191235059761 0.956 0.9540918163672655
begin training...
begin testing...
0.9224806201550387 0.952 0.937007874015748
begin training...
begin testing...
0.9053030303030303 0.956 0.9299610894941633
begin training...
begin testing...
0.9172932330827067 0.976 0.945736434108527
begin training...
begin testing...
0.8921933085501859 0.96 0.9248554913294798
begin training...
begin testing...
0.8913043478260869 0.984 0.9353612167300379
begin training...
begin testing...
0.8857142857142857 0.992 0.9358490566037736
begin training...
begin testing...
0.8825622775800712 0.992 0.9340866290018832
begin training...
begin testing...
0.8794326241134752 0.992 0.9323308270676692
begin training...
begin testing...
0.8671328671328671 0.992 0.9253731343283581
begin training...
begin testing...
0.8611111111111112 0.992 0.9219330855018587
begin training...
begin testing...
0.8611111111111112 0.99

In [161]:
import numpy as np
x = [[1,2],[3,4]]
x = np.mat(x)
w = np.array([0,1])
w*x
np.dot(w,w.T)
x.I
np.mat(np.eye(2,2,dtype=int))
x[1,1]
y = w*x
y.shape
y = np.array(y)
np.squeeze(y).shape
y[0]

array([3, 4])

In [32]:
import numpy as np
def lsClassifier(traindata,trainlabel, testdata, testlabel, Lambda):
    wordNum = len(traindata)
    trainsetLen = len(trainlabel)
    testsetLen = len(testlabel)
    #w = np.zeros(wordNum)
    print("begin training...")
    x = np.mat(traindata)
    y = np.mat(trainlabel)
    y = y.T
    #temp = np.mat(np.eye(wordNum,wordNum,dtype=int))
    w = x * (x.T)
    for i in range(wordNum):
        w[i,i] += Lambda
    w = w.I
    w = w * x
    w = w * y
    w = w.T
    print("begin testing...")
    x = np.mat(testdata)
    y = np.array(testlabel)
    ypred = w * x
    ypred = np.array(ypred)
    ypred = ypred[0]
    nPam2Pam = 0
    nPam = 0
    nNormal2Pam = 0
    for i in range(testsetLen):
        if(ypred[i]>0):
            ypred[i] = 1
        else:
            ypred[i] = -1
        if(y[i]==1):
            nPam += 1
        if(y[i]==-1 and ypred[i]==1):
            nNormal2Pam += 1
        elif(y[i]==1 and ypred[i]==1):
            nPam2Pam += 1
    SP = nPam2Pam/(nPam2Pam+nNormal2Pam)
    SR = nPam2Pam/nPam
    F = SP*SR*2/(SP+SR)
    return ypred,SP,SR,F
    
    

In [33]:
lsClassifier(traindata,trainlabel, testdata, testlabel, 1)

begin training...
begin testing...


(array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,
        -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,
         1., -1.,  1.,  1.,  1., -1.,  1.,  1., -1.,  1.,  1.,  1., -1.,
         1., -1.,  1., -1.,  1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.

In [34]:
for i in range(1,10):
    ypred,SP,SR,F = lsClassifier(traindata,trainlabel, testdata, testlabel, i)
    print(SP,SR,F)

begin training...
begin testing...
0.9705882352941176 0.924 0.9467213114754099
begin training...
begin testing...
0.9669421487603306 0.936 0.9512195121951219
begin training...
begin testing...
0.9713114754098361 0.948 0.9595141700404858
begin training...
begin testing...
0.9795081967213115 0.956 0.9676113360323886
begin training...
begin testing...
0.9795918367346939 0.96 0.9696969696969697
begin training...
begin testing...
0.9795918367346939 0.96 0.9696969696969697
begin training...
begin testing...
0.979757085020243 0.968 0.9738430583501007
begin training...
begin testing...
0.979757085020243 0.968 0.9738430583501007
begin training...
begin testing...
0.979757085020243 0.968 0.9738430583501007


In [60]:
import math
def Kernel(x1,x2,sigma):
    x1 = np.array(x1)
    x2 = np.array(x2)
    if(sigma==0):
        return x1*x2
    else:
        x1 = x1 - x2
        res = np.dot(x1, x2)
        res = - res / sigma / sigma
        res = math.exp(res)
        return res



def softsvm(traindata, trainlabel, testdata, testlabel, sigma, C):
    model = svm.SVC(kernel='linear', C=C, gamma=1) 
    if(sigma!=0):
        gamma = 1/2/sigma/sigma
        model = svm.SVC(C=C, kernel='rbf',gamma=gamma)
    X = np.array(traindata)
    y = np.array(trainlabel)
    X = np.transpose(X)
    model.fit(X, y)
    #Predict Output
    x_test = np.array(testdata)
    x_test = np.transpose(x_test)
    y_test = np.array(testlabel)
    ypred = model.predict(x_test)
    
    nPam2Pam = 0
    nNormal2Pam = 0
    nPam = 0
    for i in range(testsetLen):
        if(ypred[i]==1):
            if(y_test[i]==1):
                nPam2Pam += 1
            else:
                nNormal2Pam += 1
        if(y_test[i]==1):
            nPam += 1
    '''
    wordNum = len(traindata)
    trainsetLen = len(trainlabel)
    testsetLen = len(testlabel)
    # calculate alpha
    alpha = np.zeros(trainsetLen)
    x = np.mat(traindata)
    x = x.T
    y = np.array(trainlabel)
    grad = np.ones(trainsetLen)
    
    b = 0
    count = 0
    Sum = 0
    for i in range(trainsetLen):
        if(alpha[i]!=0):
            count += 1
            b = y[i]
            for j in range(trainsetLen):
                b = b - alpha[j]*y[j]*Kernel(x[i],x[j])
            Sum += b
    b = Sum/count
    
    testx = np.array(testdata)
    testy = np.array(testlabel)
    testx = np.transpose(x)
    ypred = np.zeros(testsetLen)
    nPam2Pam = 0
    nNormal2Pam = 0
    nPam = 0
    for i in range(testsetLen):
        ypred[i] = b
        for j in range(trainsetLen):
            ypred[i] += alpha[j]*y[j]*Kernel(x[j],testx[i])
        if(ypred[i]>0):
            ypred[i] = 1
        else:
            ypred[i] = -1
        if(ypred[i]==1):
            if(testy[i]==1):
                nPam2Pam += 1
            else:
                nNormal2Pam += 1
        if(testy[i]==1):
            nPam += 1
    '''
    SP = nPam2Pam/(nPam2Pam+nNormal2Pam)
    SR = nPam2Pam/nPam
    F = SP*SR*2/(SP+SR)
    return ypred,SP,SR,F
    

In [61]:
for C in range(1,6):
    for sigma in range(5):
        ypred,SP,SR,F = softsvm(traindata, trainlabel, testdata, testlabel, sigma, C)
        print(SP,SR,F)

0.9879518072289156 0.984 0.9859719438877755
0.8771929824561403 1.0 0.9345794392523363
0.8771929824561403 1.0 0.9345794392523363
0.8771929824561403 1.0 0.9345794392523363
0.8771929824561403 1.0 0.9345794392523363
0.9879518072289156 0.984 0.9859719438877755
0.8771929824561403 1.0 0.9345794392523363
0.8771929824561403 1.0 0.9345794392523363
0.8802816901408451 1.0 0.9363295880149813
0.8771929824561403 1.0 0.9345794392523363
0.9878542510121457 0.976 0.9818913480885312
0.8771929824561403 1.0 0.9345794392523363
0.8771929824561403 1.0 0.9345794392523363
0.8802816901408451 1.0 0.9363295880149813
0.8771929824561403 1.0 0.9345794392523363
0.9878542510121457 0.976 0.9818913480885312
0.8771929824561403 1.0 0.9345794392523363
0.8771929824561403 1.0 0.9345794392523363
0.8802816901408451 1.0 0.9363295880149813
0.8802816901408451 1.0 0.9363295880149813
0.9878542510121457 0.976 0.9818913480885312
0.8771929824561403 1.0 0.9345794392523363
0.8771929824561403 1.0 0.9345794392523363
0.8802816901408451 1.0 0

In [65]:
for C in range(1,10,2):
    for sigma in range(0,10,2):
        ypred,SP,SR,F = softsvm(traindata, trainlabel, testdata, testlabel, sigma/10, C)
        print(C, sigma/10, SP,SR,F)

1 0.0 0.9879518072289156 0.984 0.9859719438877755
1 0.2 0.8741258741258742 1.0 0.9328358208955224
1 0.4 0.8741258741258742 1.0 0.9328358208955224
1 0.6 0.8741258741258742 1.0 0.9328358208955224
1 0.8 0.8771929824561403 1.0 0.9345794392523363
3 0.0 0.9878542510121457 0.976 0.9818913480885312
3 0.2 0.8741258741258742 1.0 0.9328358208955224
3 0.4 0.8741258741258742 1.0 0.9328358208955224
3 0.6 0.8741258741258742 1.0 0.9328358208955224
3 0.8 0.8771929824561403 1.0 0.9345794392523363
5 0.0 0.9878542510121457 0.976 0.9818913480885312
5 0.2 0.8741258741258742 1.0 0.9328358208955224
5 0.4 0.8741258741258742 1.0 0.9328358208955224
5 0.6 0.8741258741258742 1.0 0.9328358208955224
5 0.8 0.8771929824561403 1.0 0.9345794392523363
7 0.0 0.9878542510121457 0.976 0.9818913480885312
7 0.2 0.8741258741258742 1.0 0.9328358208955224
7 0.4 0.8741258741258742 1.0 0.9328358208955224
7 0.6 0.8741258741258742 1.0 0.9328358208955224
7 0.8 0.8771929824561403 1.0 0.9345794392523363
9 0.0 0.9878542510121457 0.976 0

In [42]:
from sklearn import svm
model = svm.SVC(kernel='linear', C=1, gamma=1) 
# there is various option associated with it, like changing kernel, gamma and C value. Will discuss more # about it in next section.Train the model using the training sets and check score
X = np.array(traindata)
y = np.array(trainlabel)
X = np.transpose(X)
model.fit(X, y)
print(model.score(X, y))
#Predict Output
x_test = np.array(testdata)
x_test = np.transpose(x_test)
y_test = np.array(testlabel)
ypred = model.predict(x_test)


1.0
0.023333333333333334
[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1
 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1 -1 -1 -1 -1 -1  1  1 -1 -1  1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]


In [43]:
count = 0
for i in range(len(testlabel)):
    if(ypred[i]==y_test[i]):
        count += 1
print(count/len(testlabel))
print(ypred)

0.9766666666666667
[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1
 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1 -1 -1 -1 -1 -1  1  1 -1 -1  1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]


In [ ]:
wordNum = len(vocab)

filepath = "C:\\Users\\weiq6\\Desktop\\20021010_easy_ham\\easy_ham"
pathDir =  os.listdir(filepath)
len1 = len(pathDir)
filepath = "C:\\Users\\weiq6\\Desktop\\20021010_spam\\spam"
pathDir =  os.listdir(filepath)
len2 = len(pathDir)

EHdata = [[0 for i in range(len1)] for j in range(wordNum)]
Pamdata = [[0 for i in range(len2)] for j in range(wordNum)]

def parseline(line,i,spam):
    words = re.split(r'[_\W*]',line)
    for word in words:
        word = word.lower()
        if(word.isalpha() and len(word)<=15):
            if word in vocab:
                index = vocab.index(word)
                if(index>=0):
                    if(spam):
                        Pamdata[index][i] = Pamdata[index][i] + 1
                    else:
                        EHdata[index][i] = EHdata[index][i] + 1
print("processing easy ham set...")
filepath = "C:\\Users\\weiq6\\Desktop\\20021010_easy_ham\\easy_ham"
pathDir =  os.listdir(filepath)
i = 0
for allDir in pathDir:
    path = os.path.join('%s\\%s' % (filepath, allDir))
    f = open(path,"r",encoding="windows-1252")
    #print(path)
    header = True
    for line in f:
        if(line=="\n"):
            header = False
        if(header==False):
            parseline(line,i,False)
    i = i + 1
    f.close()

print("processing spam set...")
filepath = "C:\\Users\\weiq6\\Desktop\\20021010_spam\\spam"
pathDir =  os.listdir(filepath)
i = 0
for allDir in pathDir:
    path = os.path.join('%s\\%s' % (filepath, allDir))
    f = open(path,"r",encoding="windows-1252")
    header = True
    #print(path)
    try:
        for line in f:
            if(line=="\n"):
                header = False
            if(header==False):
                parseline(line,i,True)
    except:
        print("error at "+path)
    i = i + 1
    f.close()
EHdata = np.mat(EHdata)
Pamdata = np.mat(Pamdata)

In [85]:

def crossvalidation(threshold, Lambda, sigma, C, EHdata, Pamdata):
    testnum = int(len1/5)+int(len2/5)
    trainnum = len1 + len2 - testnum
    #train = np.zeros(wordNum,trainnum)
    #test = np.zeros(wordNum, testnum)
    testlabel = [1 if i < int(len1/5) else -1 for i in range(testnum)]
    trainlabel = [1 if i < len1-int(len1/5) else -1 for i in range(trainnum)]
    result1 = []
    result2 = []
    result3 = []
    for  i in range(5):
        print("fold ",i)
        begin = i*int(len1/5)
        test = EHdata[:,range(i*int(len1/5), (i+1)*int(len1/5))]
        temp = Pamdata[:,range(i*int(len2/5), (i+1)*int(len2/5))]
        test = np.hstack((test,temp))
        train = EHdata[:,range(i*int(len1/5))]
        temp = EHdata[:,range((i+1)*int(len1/5),len1)]
        train = np.hstack((train, temp))
        temp = Pamdata[:,range(i*int(len2/5))]
        train = np.hstack((train,temp))
        temp = Pamdata[:,range((i+1)*int(len2/5),len2)]
        train = np.hstack((train,temp))
        temp1 = []
        temp2 = []
        temp3 = []
        print("nBayes")
        for j in threshold:
            try:
                ypred,SP,SR,F = nBayesClassifier(train, trainlabel, test, testlabel, j)
                temp1.append(F)
                print(j,F)
            except:
                print("error at ",j)
        print("least square")
        for j in Lambda:
            try:
                ypred,SP,SR,F = lsClassifier(train,trainlabel, test, testlabel, j)
                temp2.append(F)
                print(j,F)
            except:
                print("error at ",j)
        print("soft svm")
        for j in sigma:
            for k in C:
                try:
                    ypred,SP,SR,F = softsvm(train, trainlabel, test, testlabel, j, k)
                    temp3.append(F)
                    print(j,k,F)
                except:
                    print("error at ",j,k)
        result1.append(temp1)
        result2.append(temp2)
        result3.append(temp3)
    return result1, result2, result3
# res1, res2, res3 = crossvalidation([3], [1], [0], [1], EHdata, Pamdata)

In [ ]:
r1, r2, r3 = crossvalidation([1,2,3], [0.5,1,10,100,5000], [0,100], [1,100,1000], EHdata, Pamdata)

fold  0
nBayes
begin training...
begin testing...
1 0.8884297520661156
begin training...
begin testing...
2 0.9202334630350194
begin training...
begin testing...
3 0.9232245681381958
least square
begin training...
error at  0.5
begin training...
begin testing...
1 0.9125
begin training...
error at  10
begin training...
begin testing...
100 0.9497435897435899
begin training...
begin testing...
5000 0.9758919961427195
soft svm
0 1 0.9847715736040609
0 100 0.9761092150170648
0 1000 0.9761092150170648
100 1 0.9966555183946488
100 100 0.9915966386554621
100 1000 0.9795918367346939
fold  1
nBayes
begin training...
begin testing...
1 0.9101796407185628
begin training...
begin testing...
2 0.9211295034079844
begin training...
begin testing...
3 0.928846153846154
least square
begin training...
error at  0.5
begin training...
begin testing...
1 0.8216704288939052
begin training...


In [57]:
def smoSimple(dataMatIn, classLabels, C, toler, maxIter): 
    dataMatrix = mat(dataMatIn); labelMat = mat(classLabels).transpose()
    b = 0; m,n = shape(dataMatrix)
    alphas = mat(zeros((m,1)))
    iter = 0
    while(iter<maxIter):  #迭代次数
        alphaPairsChanged=0 
        for i in range(m):  #在数据集上遍历每一个alpha
            #print alphas 
            #print labelMat
            fXi = float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[i,:].T)) + b
            #fXi=float(np.multiply(alphas, labelMat).T*dataMatrix*dataMatrix[i, :].T)+b  #.T也是转置
            Ei=fXi-float(labelMat[i]) 
            if((labelMat[i]*Ei<-toler) and (alphas[i]<C)) or ((labelMat[i]*Ei>toler) and (alphas[i]>0)): 
                j=selectJrand(i, m)  #从m中选择一个随机数，第2个alpha j
                fXj=float(multiply(alphas, labelMat).T*dataMatrix*dataMatrix[j, :].T)+b 
                Ej=fXj-float(labelMat[j]) 
                
                alphaIold=alphas[i].copy()  #复制下来，便于比较
                alphaJold=alphas[j].copy() 
                
                if(labelMat[i]!=labelMat[j]):  #开始计算L和H
                    L=max(0, alphas[j]-alphas[i]) 
                    H=min(C, C+alphas[j]-alphas[i]) 
                else: 
                    L=max(0, alphas[j]+alphas[i]-C) 
                    H=min(C, alphas[j]+alphas[i]) 
                if L==H: 
                    print('L==H') 
                    continue 
                
                #eta是alphas[j]的最优修改量，如果eta为零，退出for当前循环
                eta=2.0*dataMatrix[i, :]*dataMatrix[j, :].T-\
                    dataMatrix[i, :]*dataMatrix[i, :].T-\
                    dataMatrix[j, :]*dataMatrix[j, :].T 
                if eta>=0: 
                    print('eta>=0') 
                    continue 
                alphas[j]-=labelMat[j]*(Ei-Ej)/eta  #调整alphas[j] 
                alphas[j]=clipAlpha(alphas[j], H, L)  
                if(abs(alphas[j]-alphaJold)<0.00001):  #如果alphas[j]没有调整
                    print('j not moving enough') 
                    continue 
                alphas[i]+=labelMat[j]*labelMat[i]*(alphaJold-alphas[j])  #调整alphas[i]
                b1=b-Ei-labelMat[i]*(alphas[i]-alphaIold)*\
                    dataMatrix[i, :]*dataMatrix[i, :].T-\
                    labelMat[j]*(alphas[j]-alphaJold)*\
                    dataMatrix[i, :]*dataMatrix[j, :].T 
                b2=b-Ej-labelMat[i]*(alphas[i]-alphaIold)*\
                    dataMatrix[i, :]*dataMatrix[j, :].T-\
                    labelMat[j]*(alphas[j]-alphaJold)*\
                    dataMatrix[j, :]*dataMatrix[j, :].T 
                
                if(0<alphas[i]) and (C>alphas[i]): 
                    b=b1 
                elif(0<alphas[j]) and (C>alphas[j]): 
                    b=b2 
                else: 
                    b=(b1+b2)/2.0 
                alphaPairsChanged+=1 
                
                print('iter: %d i: %d, pairs changed %d' %(iter, i, alphaPairsChanged) )
        if(alphaPairsChanged==0): 
            iter+=1 
        else: 
            iter=0 
        print('iteration number: %d' %iter)
    return b, alphas
from numpy import *
C = 1
toler = 1
maxIter = 100000
x = np.mat(traindata)
x = x.T
smoSimple(x, trainlabel, C, toler, maxIter) 